# Data Analyzation of Rotational Experiment

## Rotational Experiment

The aim of the rotational experiment is to measure the anisotropy of speed of the light caused by the putative Lorentz violation.

Two resonators were placed on a rotational table. One is placed vertically, the other horizontally. 
If we assume that Lorentz violation exists, then the frequencies of two resonators would differ by $\delta v_0$, which has several frequency components: $2\omega_R$, $\omega_\oplus$, and $\Omega_\oplus$.  
$\omega_R$ is the rotational frequency of the experimental table. $\omega_\oplus$ is the sidereal frequency and $\Omega_\oplus$ is the annual frequency.
Coefficients in Lorentz violation theory can be obtained from these frequency components.

At each time $t$, we measure the difference of the two resonance frequencies $\delta V(t)$ (Volt) and the angle of the rotational table $\phi_R(t)$.
From $\delta V(t)$, $\phi_R(t)$, and $t$, we analysed the Lorentz violation coeffiecients through Least Square fitting of each frequency component.

Currently, we use Demodulated Least Square fitting method.

## Data analyzation using Demodulated Least Square (DLS)
### Introduction to Ordinary Least Square (OLS) Regression
If we assume that there is a linear relation between observable $y$ and independent variable $x$:
\begin{equation}
y = \beta_0 + \beta_1x_1 + \beta_2x_2
\end{equation}

Define 
$\beta = \begin{pmatrix}
\beta_0 & \beta_1 & \beta_2
\end{pmatrix}$

From data sets $Y$ and $X$, OLS gives the estimated $\beta$ :
\begin{equation}
    \beta = \frac{X^TY}{X^TX}
\end{equation}


### DLS method in rotational experiment
There are two stages in DLS method. In the first stage, we demodulate the $2\omega_R$ components using OLS. In the second stage, we extract the other components.
Compared to using OLS for all components, DLS increases the speed of analyzation. This is because most of the data is averaged in the first stage since $2\omega_R$ is the largest frequency component.

We can express $\delta v_0(t)$ as a function of $\omega_R$:
\begin{equation}
\frac{\delta v_0(t)}{v_0(t)} = A(t) + B(t)t + S(t)sin(2\omega_Rt + \phi) + C(t)cos(2\omega_Rt + \phi)
\end{equation}
while S(t) and C(t) contains rest of the frequency components.

#### 1st DLS stage
Since what we measured is $\delta V(t)$ instead of $\delta v_0(t)$, $\phi_R(t)$ instead of $\omega_R(t)$.

Followed the above equation, the relation between the $\delta V(t)$ and $\phi_R(t)$ is:
\begin{equation}
\delta V(t) = A(t) + B(t)t+ S(t)sin(2\phi_R(t)) + C(t)cos(2\phi_R(t))
\end{equation}

#### 2st DLS stage
To be continue ...


# Data Analyse Python Program
## Importing Libraries
1. numpy for manipulating data as matrix.
2. matplotlib for plotting figures
3. pandas for loading and exporting data
4. ProgressBar for showing the progress of the program

In [1]:
# This file is for conducting the 1st stage of DLS
# Zeyu 17 July 2017

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from progressbar import ProgressBar

## Functions for adding two variables with errors

In [20]:
def add_var(value1, sigma1, value2, sigma2):
    """
    This function adds two uncorrelated varibles with their values and standard error, sigmas
    It returns the total value and sigma
    """
    sigma_tot = 1 / (np.sqrt(1 / sigma1**2 + 1 / sigma2**2))
    value_tot = (value1 / sigma1**2 + value2 / sigma2**2) * sigma_tot**2
    return value_tot, sigma_tot

def sub_var(value1, sigma1, value2, sigma2):
    """
    This function subtracts two uncorrelated varibles with their values and standard error, sigmas
    'variable 1 - variable 2'.
    It returns the total value and sigma
    """
    return add_var(value1, sigma1, -value2, sigma2)

## Function for demodulating the subset
### Description of function
This function is for demodulating the $2\omega_R$ frequency component. Since the rest of the frequency is so small compared to the raw data acquisitoin frequency, the raw data is averaged through subsets. The size subset is usually determined by the number of rotations inside.

Input of the function: a subset of the raw data contains: index N, time $t$, angle $\phi_R$, and voltage $\delta V$. 

Subset structure: 
$
\begin{bmatrix}
N_i& t_i& \phi_{Ri}& \delta V_i \\
...& ... & ... & ... \\
N_j& t_j& \phi_{Rj}& \delta V_j
\end{bmatrix}
$

These variables follow:
\begin{equation}
\delta V = \beta_0 + \beta_1sin(\phi_R) + \beta_2cos(\phi_R) + \beta_3sin(2\phi_R) + \beta_4cos(2\phi_R)
\end{equation}
We ignore the linear drifting because it's too small. 
We also added $\omega_R$ component. This is because at $\omega_R$, there is a systemetic error from magnetic field of earth.

We define serveral new variables:
\begin{align}
x_0 &= 1\\
x_1 &= sin(\phi_R)\\
x_2 &= cos(\phi_R)\\
x_3 &= sin(2\phi_R)\\
x_4 &= cos(2\phi_R)
\end{align}
so that 
\begin{equation}
\delta V = \beta_0x_0 + \beta_1x_1 + \beta_2x_2 + \beta_3x_3 + \beta_4x_4.
\end{equation}

$\delta V,\ x_0,\ x_1,\ x_2,\ x_3,\ x_4$ are vectors with length $(j-i+1)$ while $\beta_0,\  \beta_1,\ \beta_2,\ \beta_3,\ \beta_4$ are fitting parameters (scales).

For simplicity, we define:
\begin{equation}
X = \begin{pmatrix}
       &    &    &    &    \\
    x0 & x1 & x2 & x3 & x4 \\
       &    &    &    &    
    \end{pmatrix}
,\ 
\beta = \begin{pmatrix}
        \beta_0 \\
        \beta_1 \\
        \beta_2 \\
        \beta_3 \\
        \beta_4
        \end{pmatrix}.
\end{equation}

So
\begin{equation}
\delta V = X \beta + \epsilon
\end{equation}
where $\epsilon$ is the error term.

### Terminology of OLS
Relation between independent variable $X$ and observation $Y$:
\begin{equation}
Y = X \beta + \epsilon
\end{equation}
where $\beta$ is the estimated parameters and $\epsilon$ is the error of the fit.

The least squares estimate gives us:
\begin{equation}
\beta = \frac{X^TY}{X^TX}
\end{equation}

Variance matrix of $\beta$ is:
\begin{equation}
V(\beta) = \frac{\sigma^2}{X^TX}
\end{equation}
where $\sigma$ is the standard deviation of the residual $\epsilon$.

The diagonal element of $V(\beta)$ can be used to calculate the standard deviation of $\beta$:
\begin{equation}
\sigma(\beta) = \sqrt{diag(V(\beta))}
\end{equation}

### Time scale of $\beta$
$\beta$ and $\sigma(\beta)$ are the function of time. But since $\beta$ contains only the frequencies of $\omega_\oplus$ and $\Omega_\oplus$, we take them to be a constant during one subset (if the subset is not too large). So the $\beta$ of each subset is set to be at the time $t_m$, which is the mean time of the subset.

### Description of function output
The output of demodulate function is a DataFrame object, params. It has four columns: beta, sigma, time, and Nsamples. 
The fitting parameters $\beta$ are stored in column beta for each subset. Their standard deviation, $\sigma(\beta)$, are stored in column sigma, the averaged time $t_m$ is stored in column time, the number of raw data in the subset is stored in column Nsamples.

Illustration of the output dataframe:


|              beta              |                     sigma                     |   time  |    Nsamples    |
|--------------------------------|-----------------------------------------------|---------|----------------|
|   [$\beta_{0}$...$\beta_{4}$]  |  [$\sigma(\beta_{0})$...$\sigma(\beta_{4})$]  |  $t_m$  |  $N_{sample}$  |


In [2]:
def demodulate(subset):
    """
    Given a subset of signal [index, timetag, angle, delta_V], fitting it by equation (stated in OLS section). 
    Return parameters include [theta, standard deviation, averaged time]
    """
    # initialization
    N = subset[:,0]
    t = subset[:,1]
    phi_R = subset[:,2]
    delta_V = subset[:,3]
    
    # creating matrix X
    X = np.zeros([subset.shape[0],5])
    X[:,0] = 1
    X[:,1] = np.sin(phi_R)
    X[:,2] = np.cos(phi_R)
    X[:,3] = np.sin(2*phi_R)
    X[:,4] = np.cos(2*phi_R)
    
    # perform OLS
    varcov = np.linalg.inv(np.dot(X.T, X))  # inverted inner product of X
    beta = np.dot(varcov, np.dot(X.T,delta_V))  # fitting parameters
    
    res = np.inner(beta.T, X) - delta_V  # residual
    dres = np.std(res)  # standard deviation of the residual
    
    V_beta = dres * dres * varcov  # variance matrix of fitting parameters
    sigma_beta = np.sqrt(np.diag(V_beta))  # standard deviation for fitting parameters
    
    # calculate the mean time for all these parameters
    t_mean = np.mean(t)  # average time for the subset
    
    # calculate the number of samples inside the subset
    N_sample = np.size(N)
    
    # packing all parameters in one vector
    params = pd.DataFrame([[beta, sigma_beta, t_mean, N_sample]],columns=['beta','sigma','time','Nsamples'])
    
    return params
    

## Function: select_param
Normally, we use demodulate function to create a set of parameters in dataframe. We do this for numbers of subsets and concatenate the output parameters as a dataframe object, *params_df*. It's not that easy to select a certain parameters from it without some trick. This function helps you to select a certain parameter, *param*, from *params_df* and convert it into a 2D array (if possible).

Based on the output of the demodulate function. *param* could be 'beta', 'sigma', 'time', 'Nsamples'.

In [3]:
def select_param(params_df, param):
    """
    Given a params dataframe, select a certain param. Output the result as 2D narray
    """
    param_df = params_df[param]  # choose a parameter
    param_na = param_df.values  # change it to narray
    param_na = np.array(param_na.tolist())  # change it to 2D narray (tricky part)
    
    return param_na

## Read Data of Rotational Experiment
All the data files are stored in the "./Data" folder. 
Currently there are two sets of data: run8 and run9. Locations of each data file are stored in the file_location.txt.

*read_data* function reads pathname of file_location.txt as an input and returns a DataFrame object contains all the data in one set.

Data structure of the raw data:

|        $N$           |                   $t$                    |       $\phi_R$       |     $\delta V$       |
|----------------------|------------------------------------------|----------------------|----------------------|
|          ...         |                   ...                    |         ...          |         ...          |

1. N is just an index of the data set.
2. $t$ is the local time of the acquisition of the data. Unit: seconds.
3. $\phi_R$ is the angle of the rotation plate. Unit: radius.
4. $\delta V$ is measured voltage from channel 2 (PLL?). Unit: V. 
5. It scales with the frequency difference.

### Time manipulation
Because we need to measure the physical quantity in an inertial frame (the sun centered frame), we need to set a standard frame for our local coordinates. This means that all the physical quantity should be calculated from this standard frame. The sidereal phase (phase regard to the sidereal rotation of the earth), $\phi_S$, is one of the quantity we are interested in:
\begin{equation}
\phi_S = \omega_\oplus t.
\end{equation}

If our local frame coincide with the standard frame when the experiment just started, $t=0$ means $\phi_S = 0$.
However, this is highly unlikely. So that we need to add an offset, $t_0$, to the local timetag t to make sure the sidereal phase are measured in the standard frame.

So we change our local time to local sidereal time $t_s$ by adding an offset:
\begin{equation}
t_s = t + t_0
\end{equation}
and
\begin{equation}
\phi_S = \omega_\oplus t_s.
\end{equation}

Since different runs started at different time, their $t_0$ are different. Micheal calculated $t_0$ for run 8 and 9 in the file: 'UTCconversionLST.xlsx'
1. run 8: $t_0$ = 4055.8764 (s)
2. run 9: $t_0$ = 1131905 + 4055.8764 (s)
3. run 10: $t_0$ = 5903354 + 4055.8764 (s)

### Voltage manipulation
What we need is the fractional frequency. So we need to transform the measured voltage to its fractional frequency.
The volatege to frequency factor is proportional to rotation frequcney. And it is 6.65726 * 1e-8 when rotattion frequcney is 360rad/s.


In [4]:
def read_data(location_file, local_time, rotation_frequency):
    """
    This function reads the raw data file, whose pathname are stored in the location_file.
    Return: DataFrame obejct that contains all the raw data.
    local_time (s) should be added to the time tag so that it starts from the local sidereal frame. 
    """
    # read locations of data file
    with open(location_file) as f:
        files = f.readlines()
    files = [x.strip() for x in files]  # remove the '\n' in each line.
    
    # read and concatenate each data file
    data_df = pd.DataFrame()  # for fast reading and storing data
    for location in files:
        data = pd.read_csv(location, sep='\t', header=None, index_col=False)
        data_df = pd.concat([data_df, data])  # concatenate each data
    
    # change time to local sidereal time
    data_df[1] = data_df[1] + local_time  
    
    # change voltage to fractional frequency [unitless]
    factor_v2f = 6.65726 * 1e-8  # calibration factor when rotation frequency is 360
    data_df[3] = data_df[3] * factor_v2f * rotation_frequency / 360
    
    return data_df

# DLS1 function
There are two stages in DLS: DLS1 and DLS2.
DLS1 recieves the raw data and number of subset_rot to demodulate and outputs the demodulated parameters.

In [6]:
def DLS1(data_na, subset_rot):
    """
    Perform the first stage of DLS. 
    data_na: raw data (numpy array).
    subset_rot: number of rotation per subset has.
    return: parameters output from DLS1 as DataFrame object
    """
    # initialization of looping parameters
    [data_rows, data_cols] = np.shape(data_na)  
    rotCounter = 0  # initialize the rotation counter
    index_start = 0  # initialize the starting index for slicing a subset

    # initialize variables for storing demodulated data
    params_df = pd.DataFrame()
    pbar = ProgressBar()  # showing progress

    # looping through row of the data 
    for i in pbar(range(1,data_rows)):
        delta_angle = data_na[i][2] - data_na[i-1][2]
        if delta_angle < -4: 
            # there is a jump of the angle, this means a rotation 
            rotCounter += 1
        if rotCounter == subset_rot:
            # calculate the subset till rotCounter reaches subset_rot
            index_end = i  # the ending index for slicing a subset ('index_end' is the first candidate of the second subset)
            subset = data_na[index_start : index_end]  # creating a subset for the demodulation (slicing from 'index_start' to 'index_end - 1')
            rotCounter = 0  # reset the counter to zero
            index_start = i  # reset the starting index 

            # demodulate the subset
            params = demodulate(subset)

            # storing demodulated parameters
            params_df = pd.concat([params_df, params])

    params_df = params_df.reset_index(drop='True')  # reset index
    return params_df

CPU times: user 21min 19s, sys: 26 s, total: 21min 45s
Wall time: 3min 37s


## Output parameters
Parameters are stored in *params_df* structured like this:  

| index |              beta              |                     sigma                     |   time  |    Nsamples    |
|-------|--------------------------------|-----------------------------------------------|---------|----------------|
|  ...  |           ...                  |                   ...                         |   ...   |      ...       |
|  433  |  [$\beta_{0}$...$\beta_{4}$]   |  [$\sigma(\beta_{0})$...$\sigma(\beta_{4})$]  |  $t_m$  |  $N_{sample}$  |
|  ...  |           ...                  |                   ...                         |   ...   |      ...       |




# DLS2

DLS2 takes the fitting parameters $\beta_3$ and $\beta_4$ from *params_df* (from DLS1) as well as the $t_m$ to calculate the coefficients of the others frequency.

Since currently, we are not using much data, we can only decorrelate $\omega_\oplus$ and $2\omega_\oplus$.

We convert local sidereal time, $t_m$, to local sidereal phase, $\phi_S$:
\begin{equation}
\phi_S = \omega_\oplus t_m.
\end{equation}

$\omega_\oplus$ is the sidereal frequency:
\begin{equation}
\omega_\oplus \approx \frac{2\pi}{23h\ 56m\ 4s}
\end{equation}

We reconstructed $\beta_3$ to *A_sin2w* in order to use demodulate function to fit the $\phi_S$ and $2\phi_S$. *A_sin2w* is structured like this:

|  $N$  |          $t_m$                 |                 $\phi_S$                      |   $\beta_3$  |
|-------|--------------------------------|-----------------------------------------------|--------------|
|  ...  |           ...                  |                   ...                         |      ...     |

Same as $\beta_4$, it's stored in *A_cos2w*.

*A_sin2w* and *A_cos2w* are fed to *demodulate* function and the coefficients of $\omega_\oplus$ and $2\omega_\oplus$ are returned.

1. *para_S* contains $S_{c, 0}$, $S_{s,\omega_\oplus}$, $S_{c,\omega_\oplus}$, $S_{s,2\omega_\oplus}$, and $S_{c,2\omega_\oplus}$, as well as their standard deviation $\sigma(S_{s,\omega_\oplus})$ ...
2. *para_C* contains $C_{c, 0}$, $C_{s,\omega_\oplus}$, $C_{c,\omega_\oplus}$, $C_{s,2\omega_\oplus}$, and $C_{c,2\omega_\oplus}$, as well as their standard deviation $\sigma(C_{s,\omega_\oplus})$ ...



After calculated two variables: para_S and para_C. We use them to calculate the SME coefficients. Please look in to Francis's lab notebook for the fomular.

In [19]:
def DLS2(params_df):
    """
    Perform 2nd stage of DLS.
    It recieves the output of DLS1 and return SME coefficients
    A list is returned. Its first element is the value of SME coefficients as a dictionary. Its second element is also a dictionary, stores the standard deviation of the SME coeffieicnet.
    """
    ## initalization
    A_cols = 4
    A_rows = params_df.shape[0]
    A = np.zeros((A_rows, A_cols))
    A[:,0] = params_df.index.values
    A[:,1] = select_param(params_df, 'time')

    w_s = 2 * np.pi / (23*3600 + 56*60 + 4)  # sidereal frequency
    A[:,2] = A[:,1] * w_s

    A_sin2w = A.copy()
    A_sin2w[:,3] = select_param(params_df, 'beta')[:,3]  # coefficeint of sin(2w) [unitless]

    A_cos2w = A.copy()
    A_cos2w[:,3] = select_param(params_df, 'beta')[:,4]  # coefficient of cos(2w) [unitless]

    ## Calculating

    para_S = demodulate(A_sin2w)
    para_C = demodulate(A_cos2w)

    ## Storing result

    DLS2_co_values = {}

    DLS2_co_values['S_c_0'] = para_S['beta'][0][0]
    DLS2_co_values['S_s_ws'] = para_S['beta'][0][1]
    DLS2_co_values['S_c_ws'] = para_S['beta'][0][2]
    DLS2_co_values['S_s_2ws'] = para_S['beta'][0][3]
    DLS2_co_values['S_c_2ws'] = para_S['beta'][0][4]

    DLS2_co_values['C_c_0'] = para_C['beta'][0][0]
    DLS2_co_values['C_s_ws'] = para_C['beta'][0][1]
    DLS2_co_values['C_c_ws'] = para_C['beta'][0][2]
    DLS2_co_values['C_s_2ws'] = para_C['beta'][0][3]
    DLS2_co_values['C_c_2ws'] = para_C['beta'][0][4]

    DLS2_co_sigma = {}

    DLS2_co_sigma['S_c_0'] = para_S['sigma'][0][0]
    DLS2_co_sigma['S_s_ws'] = para_S['sigma'][0][1]
    DLS2_co_sigma['S_c_ws'] = para_S['sigma'][0][2]
    DLS2_co_sigma['S_s_2ws'] = para_S['sigma'][0][3]
    DLS2_co_sigma['S_c_2ws'] = para_S['sigma'][0][4]

    DLS2_co_sigma['C_c_0'] = para_C['sigma'][0][0]
    DLS2_co_sigma['C_s_ws'] = para_C['sigma'][0][1]
    DLS2_co_sigma['C_c_ws'] = para_C['sigma'][0][2]
    DLS2_co_sigma['C_s_2ws'] = para_C['sigma'][0][3]
    DLS2_co_sigma['C_c_2ws'] = para_C['sigma'][0][4]

    ## Calculate EMS coefficients

    X = 58.014 * 2 * np.pi / 360  # colatitude of the laborotory (radius)
    sin_X = np.sin(X)
    cos_X = np.cos(X)

    EMS_co_values = {}
    EMS_co_sigma = {}

    EMS_co_values['c_T_Q'] = - DLS2_co_values['C_c_0'] / (4 * sin_X**2)
    EMS_co_sigma['c_T_Q'] = - DLS2_co_sigma['C_c_0'] / (4 * sin_X**2)

    EMS_co_values['c_T_Y_1'] = - DLS2_co_values['C_c_ws'] / (8 * sin_X)
    EMS_co_sigma['c_T_Y_1'] = - DLS2_co_sigma['C_c_ws'] / (8 * sin_X)
    EMS_co_values['c_T_Y_2'] = DLS2_co_values['S_s_ws'] / (8 * sin_X * cos_X)
    EMS_co_sigma['c_T_Y_2'] = DLS2_co_sigma['S_s_ws'] / (8 * sin_X * cos_X)
    EMS_co_values['c_T_Y'], EMS_co_sigma['c_T_Y'] = add_var(EMS_co_values['c_T_Y_1'], EMS_co_sigma['c_T_Y_1'],
                                                            EMS_co_values['c_T_Y_2'], EMS_co_sigma['c_T_Y_2'])

    EMS_co_values['c_T_X_1'] = - DLS2_co_values['C_s_ws'] / (8 * sin_X * cos_X)
    EMS_co_sigma['c_T_X_1'] = - DLS2_co_sigma['C_s_ws'] / (8 * sin_X * cos_X)
    EMS_co_values['c_T_X_2'] = - DLS2_co_values['S_c_ws'] / (8 * sin_X)
    EMS_co_sigma['c_T_X_2'] = - DLS2_co_sigma['S_c_ws'] / (8 * sin_X)
    EMS_co_values['c_T_X'], EMS_co_sigma['c_T_X'] = add_var(EMS_co_values['c_T_X_1'], EMS_co_sigma['c_T_X_1'],
                                                            EMS_co_values['c_T_X_2'], EMS_co_sigma['c_T_X_2'])

    EMS_co_values['c_T_Z_1'] = DLS2_co_values['C_s_2ws'] / (2 * (1 + cos_X)**2)
    EMS_co_sigma['c_T_Z_1'] = DLS2_co_sigma['C_s_2ws'] / (2 * (1 + cos_X)**2)
    EMS_co_values['c_T_Z_2'] = DLS2_co_values['S_c_2ws'] / (2 * (1 + cos_X)**2)
    EMS_co_sigma['c_T_Z_2'] = DLS2_co_sigma['S_c_2ws'] / (2 * (1 + cos_X)**2)
    EMS_co_values['c_T_Z'], EMS_co_sigma['c_T_Z'] = add_var(EMS_co_values['c_T_Z_1'], EMS_co_sigma['c_T_Z_1'],
                                                            EMS_co_values['c_T_Z_2'], EMS_co_sigma['c_T_Z_2'])

    EMS_co_values['c_T_M'], EMS_co_sigma['c_T_M'] = add_var(DLS2_co_values['C_c_2ws'], DLS2_co_sigma['C_c_2ws'], 
                                                            DLS2_co_values['S_s_2ws'], DLS2_co_sigma['S_s_2ws']) / (4 * (cos_X - 1)**2)
    EMS_co_values['c_T_u'], EMS_co_sigma['c_T_u'] = sub_var(DLS2_co_values['C_c_2ws'], DLS2_co_sigma['C_c_2ws'], 
                                                            DLS2_co_values['S_s_2ws'], DLS2_co_sigma['S_s_2ws']) / (4 * (cos_X + 1)**2)

    # change the EMS sigma to absolute 
    for key, values in EMS_co_sigma.items():
        EMS_co_sigma[key] = abs(values)
    
    return EMS_co_values, EMS_co_sigma